In [ ]:
#!/usr/bin/env python3

import os
import sys
import RPi.GPIO as GPIO
import serial
import threading
import datetime
import time
import logging
import traceback
import re
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
from tkinter import font
import queue
import csv

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    stream=sys.stdout
)

# GPIO configuration
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

gpio_pins_per_device = {
    'Port 1': {"LeftPoke": 17, "RightPoke": 27, "Pellet": 22},
    'Port 2': {"LeftPoke": 10, "RightPoke": 9, "Pellet": 11},
    'Port 3': {"LeftPoke": 0, "RightPoke": 5, "Pellet": 6},
    'Port 4': {"LeftPoke": 13, "RightPoke": 19, "Pellet": 26},
}

for device_pins in gpio_pins_per_device.values():
    for pin in device_pins.values():
        GPIO.setup(pin, GPIO.OUT)
        GPIO.output(pin, GPIO.LOW)

pellet_lock = threading.Lock()
pellet_in_well = {}
stop_event = threading.Event()
column_headers = [
    "Timestamp", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

def send_ttl_signal(pin):
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.1)
    GPIO.output(pin, GPIO.LOW)

def handle_pellet_event(event_type, port_identifier, gpio_pins, q):
    global pellet_in_well
    with pellet_lock:
        if port_identifier not in pellet_in_well:
            pellet_in_well[port_identifier] = False
        if event_type == "Pellet":
            if pellet_in_well[port_identifier]:
                GPIO.output(gpio_pins["Pellet"], GPIO.LOW)
                q.put(f"Pellet taken, signal turned OFF.")
                pellet_in_well[port_identifier] = False
                send_ttl_signal(gpio_pins["Pellet"])
            else:
                q.put(f"No pellet was in the well, no signal for pellet taken.")
        elif event_type == "PelletInWell":
            GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)
            pellet_in_well[port_identifier] = True
            q.put(f"Pellet dispensed in well, signal ON.")

def process_event(event_type, port_identifier, gpio_pins, q):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    message = f"[{timestamp}] {port_identifier} - Event: {event_type}"
    q.put(message)

    if event_type == "Left":
        send_ttl_signal(gpio_pins["LeftPoke"])
        q.put(f"{port_identifier} - Left poke event triggered.")
    elif event_type == "Right":
        send_ttl_signal(gpio_pins["RightPoke"])
        q.put(f"{port_identifier} - Right poke event triggered.")
    elif event_type in ["Pellet", "PelletInWell"]:
        handle_pellet_event(event_type, port_identifier, gpio_pins, q)

def get_device_mappings_by_usb_port():
    device_mappings = []
    usb_port_mapping = {
        'usb-0:1.1': 'Port 1',
        'usb-0:1.2': 'Port 2',
        'usb-0:1.3': 'Port 3',
        'usb-0:1.4': 'Port 4',
    }
    for symlink in os.listdir('/dev/serial/by-path/'):
        symlink_path = os.path.join('/dev/serial/by-path/', symlink)
        serial_port = os.path.realpath(symlink_path)
        if 'ttyACM' in serial_port or 'ttyUSB' in serial_port:
            usb_port_path = get_usb_port_path_from_symlink(symlink)
            port_identifier = usb_port_mapping.get(usb_port_path)
            if port_identifier:
                device_mappings.append({
                    'serial_port': serial_port,
                    'port_identifier': port_identifier,
                })
    return device_mappings

def get_usb_port_path_from_symlink(symlink):
    match = re.search(r'usb-\d+:\d+(\.\d+)*', symlink)
    return match.group() if match else None

class FED3MonitorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("HPFED DATA MONITOR V.01")
        
        # Set fullscreen for 7-inch screen
        self.root.geometry("1024x600")
        
        # Resize and arrange widgets
        self.mainframe = ttk.Frame(self.root, padding="5 5 5 5")
        self.mainframe.grid(column=0, row=0, sticky=(tk.N, tk.W, tk.E, tk.S))
        
        self.port_widgets = {}
        self.setup_ports()
        
        # Add control buttons and experiment data fields
        self.create_controls()
        
        self.root.after(100, self.update_gui)

    def setup_ports(self):
        port_names = ['Port 1', 'Port 2', 'Port 3', 'Port 4']
        for idx, port_name in enumerate(port_names):
            frame = ttk.LabelFrame(self.mainframe, text=port_name)
            frame.grid(column=idx, row=0, padx=5, pady=5, sticky=(tk.N, tk.S, tk.E, tk.W))
            status_label = ttk.Label(frame, text="Not Ready", font=("Helvetica", 12), foreground="red")
            status_label.grid(column=0, row=0, sticky=tk.W)
            text_widget = tk.Text(frame, width=30, height=10)
            text_widget.grid(column=0, row=1, sticky=(tk.N, tk.S, tk.E, tk.W))

            self.port_widgets[port_name] = {'frame': frame, 'status_label': status_label, 'text_widget': text_widget}

    def create_controls(self):
        tk.Label(self.mainframe, text="Experiment Name:", font=("Helvetica", 10)).grid(column=0, row=1, sticky=tk.W)
        self.experiment_name = ttk.Entry(self.mainframe, width=20)
        self.experiment_name.grid(column=1, row=1, sticky=tk.W)

        # Start and Stop buttons
        self.start_button = tk.Button(self.mainframe, text="START", font=("Helvetica", 12), bg="green", command=self.start_experiment)
        self.start_button.grid(column=2, row=1, padx=5)
        self.stop_button = tk.Button(self.mainframe, text="STOP", font=("Helvetica", 12), bg="red", command=self.stop_experiment)
        self.stop_button.grid(column=3, row=1, padx=5)

    def update_gui(self):
        self.root.after(100, self.update_gui)
        
    def start_experiment(self):
        # Start experiment functionality
        pass

    def stop_experiment(self):
        # Stop experiment functionality
        pass

# Main execution
if __name__ == "__main__":
    root = tk.Tk()
    app = FED3MonitorApp(root)
    root.mainloop()
